In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

n_classes = 10
batch_size = 128

learning_rate = 0.01


x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
sess = tf.InteractiveSession()

In [3]:
def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([7, 7, 1, 32])),
               'W_conv2':tf.Variable(tf.random_normal([5, 5, 32, 64])),
               'W_fc':tf.Variable(tf.random_normal([7*7*64, 1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
              'b_conv2':tf.Variable(tf.random_normal([64])),
              'b_fc':tf.Variable(tf.random_normal([1024])),
              'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv2d(x, weights['W_conv1'])
    conv1 = tf.add(conv1, biases['b_conv1'])
    conv1 = tf.nn.relu(conv1)
    conv1 = maxpool2d(conv1)
    
    conv2 = conv2d(conv1, weights['W_conv2'])
    conv2 = tf.add(conv2, biases['b_conv2'])
    conv2 = tf.nn.relu(conv2)
    conv2 = maxpool2d(conv2)
    
    fc = tf.reshape(conv2, shape=[-1, 7*7*64])
    fc = tf.nn.relu(tf.add(tf.matmul(fc, weights['W_fc']), biases['b_fc']))
    
    output = tf.add(tf.matmul(fc, weights['out']), biases['out'])
    
    return output

In [4]:
prediction = convolutional_neural_network(x)
correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
sess.run(tf.initialize_all_variables())

In [4]:
prediction = convolutional_neural_network(x)
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
optimizer = tf.train.AdamOptimizer().minimize(cost)
saver = tf.train.Saver()
sess.run(tf.initialize_all_variables())

hm_epochs = 1
for epoch in range(hm_epochs):
    epoch_loss = 0
    for _ in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
        epoch_loss += c
    print 'Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss
correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
save_path = saver.save(sess, "model.ckpt")
print "Model saved in file:", save_path

Epoch 0 completed out of 1 loss: 1871071.93256
Model saved in file: model.ckpt


In [ ]:
print 'Accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels})

Accuracy:

In [5]:
saver = tf.train.import_meta_graph("model.ckpt.meta")
saver.restore(sess, "model.ckpt")

In [ ]:
xx = mnist.train.images[4]

xr = tf.reshape(xx, shape=[28, 28])

yy =  convolutional_neural_network(xx)

sess.run(tf.initialize_all_variables())

print yy.eval()

print tf.argmin(yy*yy, 1).eval()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl

pl.imshow(xr.eval(), interpolation='none', cmap=pl.get_cmap('gray_r'))

In [ ]:
pl.show()

In [ ]:
mnist.test.labels[0]